In [ ]:
# 学習した重みから画像を生成するサンプルコード

In [1]:
# GPU情報の確認
!nvidia-smi

Sun Feb  7 12:28:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# google driveからcolaboにファイルをコピーするメソッド
import requests

def download_file_from_google_drive(id, destination):
    print('---- download_file_from_google_drive start ----')
    URL = "https://docs.google.com/uc?export=download"

    try:
        session = requests.Session()

        response = session.get(URL, params = { 'id' : id }, stream = True)
        print('response.status_code: {}'.format(response.status_code))
        print('response.headers: {}'.format(response.headers))

        token = get_confirm_token(response)
        print('token'.format(token))

        if token:
            print(token)
            params = { 'id' : id, 'confirm' : token }
            response = session.get(URL, params = params, stream = True)

        save_response_content(response, destination)
    except Exception as e:
        print(e)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        print('key: {}'.format(key))
        print('value: {}'.format(value))
       
        if key.startswith('download_warning'):
            return value
            
        if key == 'NID':
            print('NID')
            token = value.split('=')
            print(token)
            return token[1]

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
# 実行環境のrootディレクトリの確認
%cd /content
!ls -lah

In [ ]:
# ソースコードのダウンロード
%cd /content/
!rm -fr gan_sample
!git clone https://github.com/ayukat1016/gan_sample.git

In [ ]:
# 学習済みモデルを配置するディレクトリを作成します。
%cd /content/
!mkdir ./model

In [ ]:
# 学習済みモデルを下記からダウンロード
# https://drive.google.com/file/d/1FDFlJhLhwVI52ZyJHBuHalSnbJ8EcCs8/view?usp=sharing

file_id = '1FDFlJhLhwVI52ZyJHBuHalSnbJ8EcCs8'
destination = './model/20210122_070159.pkl'
download_file_from_google_drive(file_id, destination)

In [ ]:
# Google driveのマウントを行います。
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# マウントしたGoogle Drive情報の確認
!ls gdrive -lah

In [ ]:
# 画像の生成を行います
# --batch_size: 学習を行った際のバッチサイズを指定します
# --resolution: 学習を行った際の画像の解像度を指定します
# --model_path: 学習済みモデルを配置したディレクトリを指定します。
# --output_path: 生成した画像を格納するディレクトリを指定します。
# --seeds: 標準正規分布を作成する際の乱数のシードを指定します。
# seedsに指定した個数分、画像が生成され、--output_pathで指定したディレクトリ格納されます。
# --truncation-psi: 0 〜 1の間を指定します。0に近いと標準的な画像が生成され、1.0に近い値を指定すると、標準から外れた画像が生成されます。
# 0に近いと品質は良いですがつまらない画像が、1.0に近いと変わった画像が生成されますが、品質は悪くなります。
# --latent_dim: 中間潜在変数の次元数を指定します。学習時と同じ値を設定します。

%cd /content/gan_sample/chapter7/stylegan2_pytorch

!python generate_images.py \
--batch_size=4 \
--resolution=512 \
--model_path=../../../model/ \
--output_path=/content/gdrive/MyDrive/'Colab Notebooks'/gan_sample/chapter7/results \
--seeds=1,13,113,221,1059,3567,6691,10336,19440,65531 \
--truncation-psi=0.3 \
--latent_dim=512